In [1]:
from catboost import CatBoostRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import train_test_split
import pandas as pd


In [3]:

# Определение модели CatBoost
cb_model = CatBoostRegressor()

# Определение пространства поиска гиперпараметров
param_space = {
    'iterations': Integer(100, 15000),
    'learning_rate': Real(0.01, 0.5, prior='log-uniform'),
    'depth': Integer(3, 10)
}

data = pd.read_csv('DS4.csv')
dropping = ['row', 'col', 'central']
X = data.drop(dropping, axis=1)
y = data['central']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Создание объекта BayesSearchCV
bayes_search = BayesSearchCV(estimator=cb_model, search_spaces=param_space, n_iter=100, cv=5, n_jobs=-1)

# Поиск оптимальных параметров
bayes_search.fit(X_train, y_train)

# Вывод результатов
print("Best parameters:", bayes_search.best_params_)
print("Best score:", bayes_search.best_score_)


0:	learn: 0.0340660	total: 133ms	remaining: 26m 45s
1:	learn: 0.0264805	total: 135ms	remaining: 13m 34s
2:	learn: 0.0207019	total: 137ms	remaining: 9m 10s
3:	learn: 0.0164821	total: 138ms	remaining: 6m 58s
4:	learn: 0.0132629	total: 140ms	remaining: 5m 38s
5:	learn: 0.0108538	total: 142ms	remaining: 4m 45s
6:	learn: 0.0091604	total: 144ms	remaining: 4m 7s
7:	learn: 0.0078953	total: 145ms	remaining: 3m 39s
8:	learn: 0.0069949	total: 147ms	remaining: 3m 17s
9:	learn: 0.0064048	total: 149ms	remaining: 2m 59s
10:	learn: 0.0059536	total: 151ms	remaining: 2m 45s
11:	learn: 0.0056789	total: 152ms	remaining: 2m 33s
12:	learn: 0.0054408	total: 154ms	remaining: 2m 22s
13:	learn: 0.0052712	total: 155ms	remaining: 2m 14s
14:	learn: 0.0050731	total: 157ms	remaining: 2m 6s
15:	learn: 0.0049494	total: 159ms	remaining: 1m 59s
16:	learn: 0.0048521	total: 161ms	remaining: 1m 54s
17:	learn: 0.0047811	total: 162ms	remaining: 1m 48s
18:	learn: 0.0046905	total: 164ms	remaining: 1m 44s
19:	learn: 0.0046265	t

In [ ]:
from scipy.optimize import differential_evolution
from catboost import CatBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

# Создание набора данных для примера
data = pd.read_csv('DS4.csv')
dropping = ['row', 'col', 'central']
X = data.drop(dropping, axis=1)
y = data['central']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение функции для оптимизации (функция потерь)
def objective_function(params):
    # Параметры для оптимизации
    depth, learning_rate, iterations = params
    
    # Создание модели CatBoost с заданными параметрами
    catboost_model = CatBoostClassifier(depth=int(depth), learning_rate=learning_rate, iterations=int(iterations), verbose=False)
    
    # Оценка модели с использованием кросс-валидации
    scores = cross_val_score(catboost_model, X_train, y_train, cv=3, scoring='accuracy')
    
    # Минимизация отрицательного среднего качества модели (чтобы использовать оптимизацию)
    return -scores.mean()

# Определение границ параметров для оптимизации
bounds = [(4, 10), (0.01, 0.3), (1000, 15000)]  # Границы для depth, learning_rate и iterations соответственно

# Запуск алгоритма дифференциальной эволюции для оптимизации
result = differential_evolution(objective_function, bounds, maxiter=50, popsize=10)

# Получение наилучших параметров
best_params = result.x
best_score = -result.fun  # Функция потерь минимизируется, поэтому берем минус чтобы получить максимум

print("Наилучшие параметры:", best_params)
print("Наилучшая оценка:", best_score)
